In [1]:
%useLatestDescriptors
%use lets-plot

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.4.1.2-alpha1. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.3.0.0.

In [3]:
%use krangl

In [4]:
val blankTheme = theme(line=elementBlank(), axis=elementBlank())

val (w, h) = 400 to 250

val tooltipContent = layerTooltips()
                        .line("count|@{..count..} (@{..prop..})")
                        .line("total|@{..sum..}")
                        .format("..prop..", ".0%")
                        .format("..count..", ".1f")
                        .format("..sum..", ".1f")

In [5]:
fun barAndPie(
    barPlot: org.jetbrains.letsPlot.intern.Plot, 
    piePlot: org.jetbrains.letsPlot.intern.Plot
): GGBunch {
    return gggrid(
        plots = listOf(barPlot, piePlot + blankTheme),
        ncol = 2,
        cellWidth = w,
        cellHeight = h,
        fit = true
    )
}

In [6]:
val data = mapOf(
    "name" to listOf('a', 'b', 'c', 'd', 'b'),
    "value" to listOf(40, 90, 10,  50,  20)
)

val p = ggplot(data) + ggsize(w,h)

In [7]:
// Data as is - stat identity

barAndPie(
    p + geomBar(stat = Stat.identity) { x = "name"; y = "value"; fill = "name" },
    p + geomPie(stat = Stat.identity, stroke = 1) { slice = "value"; fill = "name" }
)

In [8]:
// count stat

barAndPie(
    p + geomBar() { x = "name"; fill = "name" },
    p + geomPie(stroke = 1, tooltips = tooltipContent) { fill = "name" }
)

In [9]:
// sum

barAndPie(
    p + geomBar() { x = "name"; fill = "name"; weight = "value" },
    p + geomPie(stroke = 1, tooltips = tooltipContent) { fill = "name"; weight = "value" }
)

In [10]:
// sum + ordering

barAndPie(
    p + geomBar() { x = "name"; fill = asDiscrete("name", orderBy = "..count.."); weight = "value" },
    p + geomPie(stroke = 1, tooltips = tooltipContent) { 
        fill = asDiscrete("name", orderBy = "..count..")
        weight = "value"
    }
)

In [11]:
val data2 = mapOf(
    "x" to listOf(1, 1, 1, 1, 1, 1.5, 1.5,   2,   2,   2),
    "y" to listOf(1, 1, 1, 1, 1,   2,   2, 1.5, 1.5, 1.5),
    "s" to listOf(3, 1, 2, 1, 4,   1,   3,   3,   3,  1),
    "n" to listOf('a', 'b', 'a', 'c', 'a',  'a', 'b', 'c', 'a',  'b')
)


val p2 = ggplot(data2) + xlim(0.5 to 2.5) + ylim(0.5 to 2.5)
 
p2 + geomPie(hole = 0.3, stat = Stat.identity) { x = "x"; y = "y"; slice = "s"; fill = "n" }

In [12]:
p2 + geomPie(hole = 0.3) { 
    x = "x"
    y = "y"
    fill = asDiscrete("n", orderBy = "..count..")
    weight = "s"
}

In [13]:
// Map total count in each point to size

p2 + geomPie(hole = 0.3, tooltips = tooltipContent) { 
    x = "x"
    y = "y"
    fill = asDiscrete("n", orderBy = "..count..", order = -1)
    weight = "s"
    size = "..sum.."
}

In [14]:
// Mapping 'fill' and 'size' to the same variable

ggplot(data = mapOf("n" to listOf("a", "b", "c"), "s" to listOf(1, 2, 3))) + blankTheme +
    geomPie(stat = Stat.identity) { fill = "n"; slice = "s"; size = "n" }

In [15]:
var mpgDf = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
mpgDf.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [16]:
val mpgData = mpgDf.toMap()

In [17]:
val p3 = ggplot(mpgData) + ggsize(400,300)

In [18]:
val orderedClass = asDiscrete("class", orderBy = "..count..")

barAndPie(
    p3 + geomBar() { x = "class"; fill = orderedClass },
    p3 + geomPie(hole = 0.3) { fill = orderedClass }
)

In [19]:
barAndPie(
    p3 + geomBar() { x = "class"; fill = orderedClass; weight = "displ" },
    p3 + geomPie(hole = 0.3, tooltips = tooltipContent) { fill = orderedClass; weight = "displ" }
)

In [20]:
ggplot(mpgData) + blankTheme + 
    geomPie(hole = 0.3, tooltips = tooltipContent) { fill = orderedClass; weight = "displ"; size = "..sum.." } +
    facetGrid(x = "year") +
    scaleSize(range = 5 to 10) +
    guides(size = "none")

In [21]:
ggplot(mpgData) + blankTheme + 
    geomPie(tooltips = tooltipContent) { fill = asDiscrete("drv", order = 1); size = "..sum.." } +
    facetWrap(facets = "trans", ncol = 5) +
    scaleSize(range = 2 to 10) + 
    guides(size = "none")

In [22]:
val length = mapOf(
  "name" to listOf("20-50 km", "50-75 km", "10-20 km", "75-100 km", "3-5 km", "7-10 km", "5-7 km", ">100 km", "2-3 km"),
  "count" to listOf(1109, 696, 353, 192, 168, 86, 74, 65, 53),
  "explode" to listOf(0, 0, 0, 0.1, 0.1, 0.2, 0.3, 0.4, 0.6)
)

ggplot(length) + blankTheme +
    geomPie(stat = Stat.identity, stroke = 1, strokeColor = "black", size = 20) {
        fill = "name"; slice = "count"; explode = "explode"
    } +
    scaleFillGradient(low = "dark_blue", high = "light_green")

In [23]:
val calories = mapOf(
    "slice" to listOf(35, 25, 25, 15),
    "label" to listOf("Apples", "Bananas", "Cherries", "Dates"),
    "explode" to listOf(0.1, 0, 0, 0)
)

val p4 = ggplot(calories) +
    blankTheme +
    scaleFillBrewer(palette="Set1") +
    ggsize(w, h)

gggrid(plots = listOf(
    p4 + geomPie(stat = Stat.identity, size = 18) {
        fill = "label"; slice = "slice"; explode = "explode"
    } + theme().legendPositionNone(),
    p4 + geomPie(stat = Stat.identity, size = 18, hole = 0.8) {
        fill = "label"; slice = "slice"; explode = "explode"
    }),
       ncol = 2,
       cellWidth = w,
       cellHeight = h
)